In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import timm

# ==========================================
# 🔥 딥페이크 탐지 모델 설계도 (Class)
# ==========================================
class DeepfakeDetector(nn.Module):
    def __init__(self, model_name='convnextv2_base.fcmae_ft_in22k_in1k', pretrained=True):
        super(DeepfakeDetector, self).__init__()

        print(f"🏗️ [DeepfakeDetector] 모델 생성 중... (Base: {model_name})")

        # 1. Backbone (몸통) 로드: ConvNeXt V2 Base
        # pretrained=True: 이미 똑똑한 상태로 가져옴
        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        # 2. 기존 분류기(Head) 제거 및 입력 크기 확인
        # ConvNeXt V2 Base의 경우 보통 1024개의 특징(Feature)이 나옵니다.
        if hasattr(self.backbone, 'head') and hasattr(self.backbone.head, 'fc'):
            n_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity() # 기존 Head 무력화
        else:
            # 안전장치: 혹시 구조가 다를 경우
            n_features = self.backbone.num_features
            self.backbone.reset_classifier(0)

        # 3. Custom Head (우리가 직접 튜닝하는 머리 부분)
        # 1024개 특징 -> 512 -> 128 -> 1 (가짜 확률)
        self.classifier = nn.Sequential(
            nn.Linear(n_features, 512),
            nn.BatchNorm1d(512),     # 학습 안정화
            nn.ReLU(),
            nn.Dropout(0.5),         # 과적합 방지 (중요!)

            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 1)        # 최종 결과 (0~1 사이 값 출력 예정)
        )

    def forward(self, x):
        # 이미지가 들어오면 -> 몸통 통과 -> 특징 추출 -> 내 분류기 통과 -> 결과
        features = self.backbone(x)
        output = self.classifier(features)
        return output

    # [옵션] 몸통 얼리기 기능 (처음엔 Head만 학습시킬 때 사용)
    def freeze_backbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = False
        print("❄️ Backbone(몸통)을 얼렸습니다. (Head만 학습됩니다)")

    # [옵션] 몸통 녹이기 기능 (나중에 전체 미세 조정할 때 사용)
    def unfreeze_backbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = True
        print("☀️ Backbone(몸통)을 녹였습니다. (전체 학습 가능)")

## 1. 데이터 로더(DataLoader 설정)

In [ ]:
import os
import shutil
from google.colab import files

# ==========================================
# 1. 저장 경로 설정 (로컬 SSD)
# ==========================================
local_path = '/content/local_data'
# 기존에 복사하다 만 찌꺼기가 있다면 삭제
if os.path.exists(local_path):
    shutil.rmtree(local_path)
os.makedirs(local_path, exist_ok=True)

print(f"🚀 [초고속] Kaggle에서 데이터를 코랩 SSD로 직접 다운로드합니다.")
print(f"   👉 저장 위치: {local_path}")

# ==========================================
# 2. Kaggle 인증 확인 (없으면 업로드)
# ==========================================
# kaggle.json이 없으면 다시 업로드 요청
if not os.path.exists('/root/.kaggle/kaggle.json'):
    print("\n🔑 kaggle.json 파일이 필요합니다. 파일 선택 버튼을 눌러주세요.")
    uploaded = files.upload()
    for fn in uploaded.keys():
        print(f'✅ 업로드 확인: "{fn}"')
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
else:
    print("✅ Kaggle 인증 정보가 이미 있습니다. 바로 다운로드합니다.")

# ==========================================
# 3. 다운로드 및 압축 해제
# ==========================================
print("\n⬇️ 다운로드 시작... (약 1~2분 소요)")
# -d: 데이터셋 ID, -p: 저장 경로, --unzip: 자동 압축 해제
!kaggle datasets download -d manjilkarki/deepfake-and-real-images -p {local_path} --unzip

# ==========================================
# 4. 폴더 구조 정리 (중요!)
# ==========================================
# 압축이 풀리면 'Dataset/Train' 폴더가 생깁니다.
# 우리가 원하는 경로는 '/content/local_data/Train' 입니다.

final_train_path = os.path.join(local_path, 'Train')
extracted_train_path = os.path.join(local_path, 'Dataset', 'Train')

if os.path.exists(extracted_train_path):
    print("\n📦 폴더 구조를 정리합니다...")
    # Dataset/Train -> Train 으로 이동
    shutil.move(extracted_train_path, local_path)
    # 빈 폴더 삭제
    shutil.rmtree(os.path.join(local_path, 'Dataset'))

    # Validation용 폴더가 있다면 그것도 정리 (선택사항)
    extracted_val_path = os.path.join(local_path, 'Dataset', 'Validation')
    if os.path.exists(extracted_val_path):
         shutil.rmtree(extracted_val_path) # 우린 Train만 쓸 거니까 삭제해서 용량 확보

print("\n" + "="*50)
print(f"✅ 준비 완료!")
print(f"📊 저장된 파일 확인:")
# 파일 수 세기
count = sum([len(files) for r, d, files in os.walk(final_train_path)])
print(f"   👉 총 이미지 수: {count:,}장")
print(f"   👉 경로: {final_train_path}")

🚀 [초고속] Kaggle에서 데이터를 코랩 SSD로 직접 다운로드합니다.
   👉 저장 위치: /content/local_data

🔑 kaggle.json 파일이 필요합니다. 파일 선택 버튼을 눌러주세요.


Saving kaggle.json to kaggle.json
✅ 업로드 확인: "kaggle.json"

⬇️ 다운로드 시작... (약 1~2분 소요)
Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images
License(s): unknown
 95% 1.61G/1.68G [00:01<00:00, 969MB/s]
100% 1.68G/1.68G [00:01<00:00, 1.25GB/s]

📦 폴더 구조를 정리합니다...

✅ 준비 완료!
📊 저장된 파일 확인:
   👉 총 이미지 수: 140,002장
   👉 경로: /content/local_data/Train


### {'Fake': 0, 'Real': 1} 대회는 반대이므로 나중에 추론한 다음 반전을 줘야함

In [ ]:
import torch
import random
import numpy as np
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# ==========================================
# 0. 시드(Seed) 고정 함수
# ==========================================
def set_seed(seed=42):
    """
    모든 랜덤 변수를 고정하여, 실행할 때마다 같은 결과가 나오도록 함
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # 멀티 GPU 사용 시
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"🔒 모든 랜덤 시드가 {seed}번으로 고정되었습니다.")

# 시드 고정 실행 (가장 먼저 호출!)
set_seed(42)

# ==========================================
# 1. 설정 (Configuration)
# ==========================================
BATCH_SIZE = 32
IMG_SIZE = 224
# 🔥 [중요] 구글 드라이브가 아니라, 아까 복사해둔 '로컬(SSD)' 경로를 사용합니다.
# (속도가 20배 빨라집니다)
DATA_PATH = '/content/local_data/Train'

# ==========================================
# 2. 이미지 전처리 (Transforms) - 수정됨
# ==========================================
transform = transforms.Compose([
    # 1. 비율 유지하며 리사이즈 (짧은 변 기준 256px)
    transforms.Resize(256),

    # 2. 중앙 크롭 (얼굴 비율 왜곡 없이 224x224 추출)
    transforms.CenterCrop(IMG_SIZE),

    # 3. 데이터 증강 (Augmentation)
    transforms.RandomHorizontalFlip(p=0.5), # 좌우 반전
    transforms.RandomRotation(10),          # 살짝 회전

    # 4. 텐서 변환 및 정규화
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ==========================================
# 3. 데이터셋 로드 및 분할 (8:2)
# ==========================================
print(f"📂 데이터 로드 경로: {DATA_PATH}")

# 로컬 경로에 파일이 진짜 있는지 안전장치 확인
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"🚨 오류: {DATA_PATH} 경로가 없습니다. \n👉 직전에 알려드린 '데이터 복사 코드(!cp)'를 먼저 실행해주세요!")

full_dataset = datasets.ImageFolder(root=DATA_PATH, transform=transform)

# 전체 데이터 개수
total_size = len(full_dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size

# 무작위 분할 (위에서 시드를 고정했으므로 매번 똑같이 나뉨)
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# 데이터 로더 생성
# num_workers=2: 로컬 SSD에서는 2~4 정도면 충분히 빠릅니다.
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print("-" * 50)
print(f"✅ 데이터 로드 및 전처리 완료!")
print(f"   👉 학습 데이터(Train): {len(train_dataset):,}장")
print(f"   👉 검증 데이터(Val):   {len(val_dataset):,}장")
print(f"   👉 클래스 맵핑: {full_dataset.class_to_idx}")
# 결과 예시: {'Fake': 0, 'Real': 1}
print("-" * 50)

🔒 모든 랜덤 시드가 42번으로 고정되었습니다.
📂 데이터 로드 경로: /content/local_data/Train
--------------------------------------------------
✅ 데이터 로드 및 전처리 완료!
   👉 학습 데이터(Train): 112,001장
   👉 검증 데이터(Val):   28,001장
   👉 클래스 맵핑: {'Fake': 0, 'Real': 1}
--------------------------------------------------


## 2. 학습 및 검증 함수(Trainer)

In [ ]:
import torch.optim as optim
from tqdm import tqdm

# 손실 함수: 이진 분류(Binary Classification)이므로 BCEWithLogitsLoss 사용
criterion = torch.nn.BCEWithLogitsLoss()

def train_one_epoch(model, loader, optimizer, device):
    model.train() # 학습 모드 전환
    running_loss = 0.0
    correct = 0
    total = 0

    loop = tqdm(loader, leave=True) # 진행바 표시

    for images, labels in loop:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

        # 1. 예측 (Forward)
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 2. 역전파 (Backward)
        optimizer.zero_grad() # 이전 기울기 초기화
        loss.backward()       # 기울기 계산
        optimizer.step()      # 가중치 업데이트

        # 3. 통계 계산
        running_loss += loss.item()

        # 확률이 0.5 이상이면 1(Real/Fake), 아니면 0으로 판단
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # 진행바에 실시간 정확도 표시
        loop.set_description(f"Train Loss: {loss.item():.4f}")

    avg_loss = running_loss / len(loader)
    acc = 100 * correct / total
    return avg_loss, acc

def evaluate(model, loader, device):
    model.eval() # 평가 모드 (Dropout 꺼짐)
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad(): # 평가 땐 기울기 계산 X (메모리 절약)
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(loader)
    acc = 100 * correct / total
    return avg_loss, acc

## 3. 메인 실행

In [ ]:
# ==========================================
# 🏁 학습 실행 (Main Loop)
# ==========================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 1. 모델 준비
model = DeepfakeDetector().to(device)

print("\n" + "="*60)
print("❄️ [Step 1] 백본 얼리기 학습 (Head Warming-up)")
print("="*60)

# 백본 얼리기
model.freeze_backbone()

# Optimizer: Head만 학습하므로 학습률(lr)을 조금 크게(1e-3) 잡음
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

# Step 1 학습 (짧게 1~3 에폭만)
epochs_step1 = 2

for epoch in range(epochs_step1):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, device)
    val_loss, val_acc = evaluate(model, val_loader, device)

    print(f"Epoch [{epoch+1}/{epochs_step1}] "
          f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%  ||  "
          f"Val Loss: {val_loss:.4f} | Acc: {val_acc:.2f}%")


print("\n" + "="*60)
print("☀️ [Step 2] 백본 녹이기 & 전체 미세조정 (Fine-Tuning)")
print("="*60)

# 백본 녹이기
model.unfreeze_backbone()

# Optimizer: 전체를 학습하므로 학습률을 아주 작게(1e-4 ~ 1e-5) 줄여야 함! (중요)
# 그래야 박사님(백본)의 지식이 깨지지 않습니다.
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Step 2 학습 (길게 5~10 에폭)
epochs_step2 = 5

for epoch in range(epochs_step2):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, device)
    val_loss, val_acc = evaluate(model, val_loader, device)

    print(f"Epoch [{epoch+1}/{epochs_step2}] "
          f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%  ||  "
          f"Val Loss: {val_loss:.4f} | Acc: {val_acc:.2f}%")

    # 모델 저장 (성능이 좋을 때마다 저장 추천)
    torch.save(model.state_dict(), f"deepfake_model_epoch_{epoch+1}.pth")

print("\n🎉 모든 학습이 완료되었습니다!")

🏗️ [DeepfakeDetector] 모델 생성 중... (Base: convnextv2_base.fcmae_ft_in22k_in1k)

❄️ [Step 1] 백본 얼리기 학습 (Head Warming-up)
❄️ Backbone(몸통)을 얼렸습니다. (Head만 학습됩니다)


Train Loss: 0.3964: 100%|██████████| 3500/3500 [05:40<00:00, 10.27it/s]


Epoch [1/2] Train Loss: 0.3748 | Acc: 83.09%  ||  Val Loss: 0.3138 | Acc: 86.53%


Train Loss: 0.2620: 100%|██████████| 3500/3500 [05:40<00:00, 10.29it/s]


Epoch [2/2] Train Loss: 0.3343 | Acc: 85.35%  ||  Val Loss: 0.3022 | Acc: 87.16%

☀️ [Step 2] 백본 녹이기 & 전체 미세조정 (Fine-Tuning)
☀️ Backbone(몸통)을 녹였습니다. (전체 학습 가능)


...

Epoch [3/5] Train Loss: 0.0305 | Acc: 98.84%  ||  Val Loss: 0.0475 | Acc: 98.46%


Train Loss: 0.0042: 100%|██████████| 3500/3500 [21:01<00:00,  2.78it/s]


Epoch [4/5] Train Loss: 0.0252 | Acc: 98.99%  ||  Val Loss: 0.0319 | Acc: 98.85%


Train Loss: 0.0261: 100%|██████████| 3500/3500 [21:01<00:00,  2.78it/s]


Epoch [5/5] Train Loss: 0.0203 | Acc: 99.20%  ||  Val Loss: 0.0477 | Acc: 98.79%

🎉 모든 학습이 완료되었습니다!


## 4. 학습된 모델 저장

In [ ]:
import shutil
import os
import glob

# 1. 옮길 파일들이 있는 곳 (현재 코랩 임시 위치)
source_dir = '/content'
# 2. 저장할 구글 드라이브 위치
target_dir = '/content/drive/MyDrive/DILAB/OK/hecto-deepfake-detection/models'

# 폴더가 없으면 생성
os.makedirs(target_dir, exist_ok=True)

print(f"🚚 모델 구조 작전을 시작합니다...")
print(f"   목적지: {target_dir}")

# 'deepfake_model_epoch_'로 시작하는 모든 .pth 파일 찾기
pth_files = glob.glob(os.path.join(source_dir, "deepfake_model_epoch_*.pth"))

if len(pth_files) == 0:
    print("❌ 파일을 못 찾았습니다! 혹시 이미 지워졌거나 이름이 다를 수 있습니다.")
else:
    for file_path in pth_files:
        file_name = os.path.basename(file_path)
        dest_path = os.path.join(target_dir, file_name)

        # 파일 이동 (move) 또는 복사 (copy)
        shutil.copy(file_path, dest_path)
        print(f"   ✅ 저장 완료: {file_name}")

print("\n🎉 모든 모델을 구글 드라이브로 대피시켰습니다! 드라이브 폴더를 확인해보세요.")

🚚 모델 구조 작전을 시작합니다...
   목적지: /content/drive/MyDrive/DILAB/OK/hecto-deepfake-detection/models
   ✅ 저장 완료: deepfake_model_epoch_5.pth
   ✅ 저장 완료: deepfake_model_epoch_2.pth
   ✅ 저장 완료: deepfake_model_epoch_4.pth
   ✅ 저장 완료: deepfake_model_epoch_1.pth
   ✅ 저장 완료: deepfake_model_epoch_3.pth

🎉 모든 모델을 구글 드라이브로 대피시켰습니다! 드라이브 폴더를 확인해보세요.


# 검증

## 1. Valdiation 데이터 복구 (다운로드)

In [ ]:
import os
import shutil

# ==========================================
# 1. 경로 설정
# ==========================================
local_path = '/content/local_data'
val_target_path = os.path.join(local_path, 'Validation')

# 혹시 찌꺼기가 있다면 삭제
if os.path.exists(val_target_path):
    shutil.rmtree(val_target_path)

print(f"🚀 [검증 데이터 확보] Kaggle에서 Validation 데이터를 가져옵니다.")

# ==========================================
# 2. 다운로드 (이미 있으면 캐시 사용됨)
# ==========================================
# Kaggle 데이터셋 다시 다운로드 (Train은 덮어써도 상관없음)
!kaggle datasets download -d manjilkarki/deepfake-and-real-images -p {local_path} --unzip

# ==========================================
# 3. Validation 폴더 구출 작전
# ==========================================
# 압축 풀린 원본 경로: /content/local_data/Dataset/Validation
extracted_val_path = os.path.join(local_path, 'Dataset', 'Validation')

if os.path.exists(extracted_val_path):
    print("\n📦 Validation 폴더를 이동합니다...")
    # Dataset/Validation -> local_data/Validation 으로 이동
    shutil.move(extracted_val_path, local_path)

    # 이제 필요 없는 껍데기(Dataset) 폴더는 삭제
    if os.path.exists(os.path.join(local_path, 'Dataset')):
        shutil.rmtree(os.path.join(local_path, 'Dataset'))

    print(f"✅ 이동 완료!")

    # 파일 개수 확인
    count = sum([len(files) for r, d, files in os.walk(val_target_path)])
    print(f"📊 검증용 이미지 수: {count:,}장")
    print(f"👉 경로: {val_target_path}")
else:
    print("⚠️ 오류: Validation 폴더를 찾을 수 없습니다. 다운로드를 확인하세요.")

🚀 [검증 데이터 확보] Kaggle에서 Validation 데이터를 가져옵니다.
Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images
License(s): unknown
 94% 1.59G/1.68G [00:02<00:00, 244MB/s] 
100% 1.68G/1.68G [00:04<00:00, 439MB/s]

📦 Validation 폴더를 이동합니다...
✅ 이동 완료!
📊 검증용 이미지 수: 39,428장
👉 경로: /content/local_data/Validation


## 2. 검증 실행

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm
import os

# ==========================================
# 1. 설정 (Epoch 4 모델 사용)
# ==========================================
BATCH_SIZE = 32
IMG_SIZE = 224
VAL_PATH = '/content/local_data/Validation'
# 아까 저장한 Epoch 4 모델 경로
MODEL_PATH = '/content/drive/MyDrive/DILAB/OK/hecto-deepfake-detection/models/deepfake_model_epoch_4.pth'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ==========================================
# 2. 모델 구조 정의 (학습 때와 똑같이!)
# ==========================================
class DeepfakeDetector(nn.Module):
    def __init__(self, model_name='convnextv2_base.fcmae_ft_in22k_in1k', pretrained=False):
        super(DeepfakeDetector, self).__init__()
        # 껍데기만 만들 거라 pretrained=False로 해서 다운로드 시간 절약
        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if hasattr(self.backbone, 'head') and hasattr(self.backbone.head, 'fc'):
            n_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
        else:
            n_features = self.backbone.num_features
            self.backbone.reset_classifier(0)

        self.classifier = nn.Sequential(
            nn.Linear(n_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        features = self.backbone(x)
        return self.classifier(features)

# ==========================================
# 3. 데이터 로드 (정직한 평가)
# ==========================================
# 검증용 전처리: 회전/반전 금지! 오직 크기 조절만
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

print(f"🚀 검증 데이터를 로드합니다...")
if os.path.exists(VAL_PATH):
    real_val_dataset = datasets.ImageFolder(root=VAL_PATH, transform=val_transform)
    real_val_loader = DataLoader(real_val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    print(f"   📂 데이터 개수: {len(real_val_dataset):,}장")
    print(f"   🏷️ 클래스 맵핑: {real_val_dataset.class_to_idx}")
else:
    print(f"❌ 오류: 경로가 없습니다 ({VAL_PATH})")
    exit()

# ==========================================
# 4. 모델 로드 및 평가
# ==========================================
print(f"\n🤖 모델을 불러옵니다: {os.path.basename(MODEL_PATH)}")
model = DeepfakeDetector().to(device)

if os.path.exists(MODEL_PATH):
    model.load_state_dict(torch.load(MODEL_PATH))
    print("   ✅ 가중치(Weights) 로드 성공!")
else:
    print("   ❌ 모델 파일이 없습니다! 경로를 확인하세요.")
    exit()

# 평가 시작
model.eval()
correct = 0
total = 0

print("\n🎯 채점 시작 (잠시만 기다려주세요)...")
with torch.no_grad():
    for images, labels in real_val_loader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

        outputs = model(images)
        predicted = (torch.sigmoid(outputs) > 0.5).float()

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

acc = 100 * correct / total

print("="*50)
print(f"🏆 최종 성적표 (Validation Set)")
print(f"   정확도 (Accuracy): {acc:.2f}%")
print("="*50)

# 결과 해석
if acc >= 90:
    print("🎉 대박! 진짜 실력입니다. 모델이 완벽하게 학습되었습니다.")
elif acc >= 70:
    print("👍 준수합니다. 실전에서도 쓸만한 모델입니다.")
elif acc >= 50:
    print("🤔 아쉽네요. 과적합(Overfitting)이 의심됩니다. (찍기 수준)")
else:
    print("😭 뭔가 잘못되었습니다.")

🚀 검증 데이터를 로드합니다...
   📂 데이터 개수: 39,428장
   🏷️ 클래스 맵핑: {'Fake': 0, 'Real': 1}

🤖 모델을 불러옵니다: deepfake_model_epoch_4.pth
   ✅ 가중치(Weights) 로드 성공!

🎯 채점 시작 (잠시만 기다려주세요)...
🏆 최종 성적표 (Validation Set)
   정확도 (Accuracy): 98.62%
🎉 대박! 진짜 실력입니다. 모델이 완벽하게 학습되었습니다.
